In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from glow import waveform
from glow import time_domain_c, freq_domain_c, freq_domain_c
from glow import physical_units as phys
from glow import tools

import astropy.units as u
import astropy.constants as c
import astropy.cosmology.units as cu
from astropy.cosmology import Planck18 as cosmo

GMsun8pi=(8*c.G*c.M_sun*np.pi/c.c**3).decompose().value
yr_to_s=u.yr.to(u.s)


In [ ]:
# Produce unlensed wavefrom
detector='LISA'
z_src_waveform= 5 * cu.redshift
dL_src= z_src_waveform.to(u.Mpc, cu.with_redshift(cosmo, distance="luminosity")).value
Mtot=1e7
Mtot_detector= Mtot*(1+z_src_waveform)
q=1
spin=0
inc=0
Tobs=(0.8*u.yr).to('s').value
# Lower bound of the frequecy. Maximum between expected frequency at -Tobs and the detector observable bound.
f_lower= np.amax([waveform.f0_obs(Mtot_detector, Tobs),waveform.f_bounds_detector(detector)[0]])
f_final= waveform.f_isco(Mtot_detector)*(1+z_src_waveform) # If too short, multiply by k>1

# Same keys of get_fd_waveform in pycbc

params_source= {'approximant': "IMRPhenomXHM",
            'mass1'          : Mtot_detector * q/(1. + q),
            'mass2'          : Mtot_detector * 1/(1. + q),
            'spin1z'         : spin,
            'spin2z'         : spin,
            'distance'       : dL_src,
            'inclination'    : inc,
            'long_asc_nodes' : 0,
            'f_lower'        : f_lower,
            'delta_f'        : 1/Tobs,
            'f_final':  f_final
            }


# Generate Frequency Domain (FD) Waveform object

h_fd=waveform.WaveformFD(params_source)


In [ ]:
# Sky location and polariation can be specified in the params_source (above) or later (below)
# Once specified, the projected strain is computed 

sky_dict = {'declination' : 0.05, 'right_ascension' : 3.67, 'polarization': 0.34}

h_fd.get_projected_strain(sky_dict=sky_dict)


In [ ]:
# If needed, load a psd function either from file or pycbc (get_psd_from_file/_pycbc)
# External psd can be used, check documentation

psd=waveform.get_psd_from_file(detector)
h_fd.load_psd(psd)

In [ ]:
plt.loglog(h_fd.sample_frequencies,h_fd.sample_frequencies*np.abs(h_fd.strain), label='h strain')
plt.loglog(h_fd.sample_frequencies,np.sqrt(h_fd.sample_frequencies*np.abs(h_fd.psd_grid)), 'k',label='sensitivity')
plt.ylabel('Characteristic strain')
plt.legend()
print('SNR of the signal: {:.1f}'.format(h_fd.snr))

In [ ]:
# Produce and plot the time-domain waveform
# We shift by the ringdown duration

h_td = h_fd.to_timedomain(cyclic_time_shift=-Mtot/300)
hp_td, hx_td= h_td.polarizations

plt.plot(hp_td.sample_times, h_td.strain)
plt.xlim(-Tobs/100,Mtot/300)

## Lensed Waveforms

Here we show how to compute lensed waveforms starting from the above unlensed waveform.

### Example 1: weak Lensing

In [ ]:
# We first define the lens and units objects

zl = 1.1
zs = 3.

p_lens = {'name':'CIS', 'Mvir_Msun':8e9, 'rc_pc':10}

Psi_lens, units_lens = phys.Lens_Units(zl, zs, p_lens)

Psi_lens.display_info()

In [ ]:
# We compute the lensing amplification factor in time and frequency domain using GLOW

y=2

It = time_domain_c.It_SingleIntegral_C(Psi_lens, y)

Fw= freq_domain_c.Fw_FFT_C(It, p_prec={'wmin':1})


In [ ]:
# Now combined Fw and the units with the unlensed waveform to get the lensed one
# When w_opt=True Fw is recomputed within the optimal frequency set by the waveform's sampling frequency range, if it's not already the case. A warning informs the user.

h_fd_lens= waveform.get_lensed_fd_from_Fw(h_fd, Fw, units_lens, w_opt=True)


In [ ]:
# We can check if a waveform object was lensed:

if h_fd_lens.islensed:
    print('This is a lensed waveform!')

# The lensed waveform object holds the unlensed waveform:

h_fd_lens.unlensed


In [ ]:
freqs=h_fd.sample_frequencies
plt.loglog(freqs, freqs*np.abs(h_fd.strain), '-', c='C0', label='unlensed')
plt.loglog(freqs, freqs*np.abs(h_fd_lens.strain), c='C1', label='lensed')
plt.loglog(freqs, np.sqrt(freqs*h_fd.psd_grid), '-', c='k', alpha=0.5, label='sensitivity')


plt.xlabel('f [Hz]')
plt.ylabel('Characteristic strain')

plt.xlim(h_fd.low_frequency_cutoff,h_fd.high_frequency_cutoff)

# plt.ylim(top=1e-15, bottom=5e-22)
plt.legend()
plt.grid()

In [ ]:
# Time domain waveforms 

h_td_lens = h_fd_lens.to_timedomain(cyclic_time_shift=-Mtot/300, unlensed=True) # If unlensed=True also the unlensed waveform is transformed and included in the time domain waveform object  
hp_td_lens, hx_td_lens= h_td_lens.polarizations

h_td = h_td_lens.unlensed
hp_td, hx_td = h_td.polarizations

ts=h_td.sample_times.numpy()

In [ ]:
fig, ax= plt.subplots(figsize=(10,5))

ax.tick_params(axis='x',          # changes apply to the x-axis
                which='both',      # both major and minor ticks are affected
                bottom=False,      # ticks along the bottom edge are off
                top=False,         # ticks along the top edge are off
                labelbottom=False)

ax.plot(ts, hp_td, '-', label='unlensed', c='C0', alpha=0.6)
ax.plot(ts, hp_td_lens, label='lensed', c='C1', zorder=0)
ax.set_ylabel('\'+\' polarization')

xlim=(-Tobs/100,Mtot/300)
ax.set_xlim(*xlim)
ax.set_xlabel('t [s]')
ax.grid(alpha=0.4)

ax.set_title('$M_{{\\rm BBH}}= {:s}\,M_\odot,\,M_{{\\rm lz}}={:s}\,M_\odot,\,y={:.1f}$'.format(tools.latex_float(Mtot),tools.latex_float(units_lens.Mlz.value),y))

ax.legend(ncols=4)



In [ ]:
# Normalizing the strain by the SNR we single out the pure wave optics distortions, besides overall magnifications.

fig, ax= plt.subplots(figsize=(10,5))

ax.tick_params(axis='x',          # changes apply to the x-axis
                which='both',      # both major and minor ticks are affected
                bottom=False,      # ticks along the bottom edge are off
                top=False,         # ticks along the top edge are off
                labelbottom=False)

ax.plot(ts, h_td.strain/h_fd.snr, '-', label='unlensed', c='C0', alpha=0.6)
ax.plot(ts, h_td_lens.strain/h_fd_lens.snr, '-', label='lensed', c='C1', zorder=0)

ax.set_ylabel('strain')

xlim=(-Tobs/50,Mtot/300)
ax.set_xlim(*xlim)
ax.set_xlabel('t [s]')
ax.grid(alpha=0.4)

ax.set_title('$M_{{\\rm BBH}}= {:s}\,M_\odot,\,M_{{\\rm lz}}={:s}\,M_\odot,\,y={:.1f}$'.format(tools.latex_float(Mtot),tools.latex_float(units_lens.Mlz.value),y))

ax.legend(ncols=4)

### Example 2: strong lensing

In [ ]:
# Produce unlensed wavefrom

detector='LIGO'
z_src_waveform= 0.3 * cu.redshift
dL_src= z_src_waveform.to(u.Mpc, cu.with_redshift(cosmo, distance="luminosity")).value
Mtot=100
Mtot_detector= Mtot*(1+z_src_waveform)
q=1
spin=0
inc=0
Tobs=(50*u.s).value
# Lower bound of the frequecy. Maximum between expected frequency at -Tobs and the detector observable bound.
f_lower= np.amax([waveform.f0_obs(Mtot_detector, Tobs),waveform.f_bounds_detector(detector)[0]])
f_final= 5*waveform.f_isco(Mtot_detector)*(1+z_src_waveform) # If too short, multiply by k>1

# Same keys of get_fd_waveform in pycbc

params_source= {'approximant': "IMRPhenomXHM",
            'mass1'          : Mtot_detector * q/(1. + q),
            'mass2'          : Mtot_detector * 1/(1. + q),
            'spin1z'         : spin,
            'spin2z'         : spin,
            'distance'       : dL_src,
            'inclination'    : inc,
            'long_asc_nodes' : 0,
            'f_lower'        : f_lower,
            'delta_f'        : 1/Tobs,
            'f_final':  f_final
            }

sky_dict = {'declination' : 0.05, 'right_ascension' : 3.67, 'polarization': 0.34}

params_source.update(sky_dict)

# Generate Frequency Domain (FD) Waveform object

h_fd=waveform.WaveformFD(params_source)


In [ ]:
# If needed, load a psd function either from file or pycbc (get_psd_from_file/_pycbc)
# External psd can be used, check documentation

psd=waveform.get_psd_from_file(detector)

h_fd.load_psd(psd)

In [ ]:
plt.loglog(h_fd.sample_frequencies,h_fd.sample_frequencies*np.abs(h_fd.strain), label='h strain')
plt.loglog(h_fd.sample_frequencies,np.sqrt(h_fd.sample_frequencies*np.abs(h_fd.psd_grid)), 'k',label='sensitivity')
plt.ylabel('Characteristic strain')
plt.legend()
print('SNR of the signal: {:.1f}'.format(h_fd.snr))

In [ ]:
# Time-domain waveform
h_td = h_fd.to_timedomain(alpha=0.04, cyclic_time_shift=-Mtot/300) # alpha tunes the windowing of the signal in the inverse fft
hp_td, hx_td= h_td.polarizations

plt.plot(hp_td.sample_times, hp_td)
# plt.ylim(-1e-25,1e-25)
# plt.xlim(-28,-20)

plt.xlim(-Tobs/50, Mtot/300)

In [ ]:
# We first define the lens and units objects

zl = z_src_waveform/2
zs = z_src_waveform

p_lens = {'name':'point lens', 'M_Msun':1e4}

Psi_lens, units_lens = phys.Lens_Units(zl, zs, p_lens)

Psi_lens.display_info()

In [ ]:
# Now combine Fw and the units with the unlensed waveform to get the lensed one
# When w_opt=True Fw is recomputed within the optimal frequency set by the waveform's sampling frequency range, if it's not already the case. A warning informs the user.

y=3 # This correspond to multiple, non-overlapping, images

Fw= freq_domain_c.Fw_AnalyticPointLens_C(y)

h_fd_lens= waveform.get_lensed_fd_from_Fw(h_fd, Fw, units_lens, w_opt=True)

In [ ]:
# Compute the time delay between images 

if len(Fw.It.p_crits)>1:
    print('Multiple images found')
else:
    print('Single image regime')

Dtau=Fw.It.p_crits[1]['t']-Fw.It.p_crits[0]['t']
time_delay=GMsun8pi/(2*np.pi)*units_lens.Mlz.to('Msun').value*Dtau

print('Time delay: {:s} s'.format(tools.latex_float(time_delay)))

In [ ]:
freqs=h_fd.sample_frequencies
plt.loglog(freqs, freqs*np.abs(h_fd.strain), '-', c='C0', label='unlensed')
plt.loglog(freqs, freqs*np.abs(h_fd_lens.strain), c='C1', label='lensed')
plt.loglog(freqs, np.sqrt(freqs*h_fd.psd_grid), '-', c='k', label='sensitivity')


plt.xlabel('f [Hz]')
plt.ylabel('Characteristic strain')

plt.xlim(h_fd.low_frequency_cutoff, h_fd.high_frequency_cutoff)

plt.legend()
plt.grid()

In [ ]:
# Time domain waveforms 
# We shift by the approx time delay of the second image + the ringdown duration

shift=-time_delay-Mtot/300

h_td_lens = h_fd_lens.to_timedomain(alpha=0.04, cyclic_time_shift=shift, unlensed=True)
hp_td_lens, hx_td_lens= h_td_lens.polarizations

h_td = h_td_lens.unlensed
hp_td, hx_td = h_td.polarizations

ts=h_td.sample_times.numpy()

In [ ]:
fig, ax= plt.subplots(figsize=(10,5))


ax.plot(ts, h_td_lens.strain, label='lensed', c='C1')
ax.set_ylabel('strain')
xlim=(-Mtot/50, time_delay+Mtot/300)
ax.set_xlim(*xlim)

ax.set_xlabel('t [s]')
ax.grid(alpha=0.4)

ax.set_title('$M_{{\\rm BBH}}= {:s}\,M_\odot,\,M_{{\\rm lz}}={:s}\,M_\odot,\,y={:.1f}$'.format(tools.latex_float(Mtot),tools.latex_float(units_lens.Mlz.value),y))

ax.legend(ncols=4)



### Example 3: micro/milli-lensing

In [ ]:
# We first define the lens and units objects

zl = z_src_waveform/2
zs = z_src_waveform

p_lens = {'name':'point lens', 'M_Msun':4e2}

Psi_lens, units_lens = phys.Lens_Units(zl, zs, p_lens)

Psi_lens.display_info()

In [ ]:
# Using the unlensed waveform introduced in the section above, we now show the case of images interference

y=1 # Overlapping images

Fw= freq_domain_c.Fw_AnalyticPointLens_C(y)

h_fd_lens= waveform.get_lensed_fd_from_Fw(h_fd, Fw, units_lens, w_opt=True)

In [ ]:
# Compute the time delay between images 

if len(Fw.It.p_crits)>1:
    print('Multiple images found')
else:
    print('Single image regime')

Dtau=Fw.It.p_crits[1]['t']-Fw.It.p_crits[0]['t']
time_delay=GMsun8pi/(2*np.pi)*units_lens.Mlz.to('Msun').value*Dtau

print('Time delay: {:s} s'.format(tools.latex_float(time_delay)))

In [ ]:
freqs=h_fd.sample_frequencies
plt.loglog(freqs, freqs*np.abs(h_fd.strain), '-', c='C0', label='unlensed')
plt.loglog(freqs, freqs*np.abs(h_fd_lens.strain), c='C1', label='lensed')
plt.loglog(freqs, np.sqrt(freqs*h_fd.psd_grid), '-', c='k', label='sensitivity')


plt.xlabel('f [Hz]')
plt.ylabel('Characteristic strain')

plt.xlim(h_fd.low_frequency_cutoff, h_fd.high_frequency_cutoff)

plt.legend()
plt.grid()

In [ ]:
# Time domain waveforms 
# We shift by the approx time delay of the second image + the ringdown duration

shift=-time_delay-Mtot/300

h_td_lens = h_fd_lens.to_timedomain(alpha=0.04, cyclic_time_shift=shift, unlensed=True)
hp_td_lens, hx_td_lens= h_td_lens.polarizations

h_td = h_td_lens.unlensed
hp_td, hx_td = h_td.polarizations

ts=h_td.sample_times.numpy()

In [ ]:
fig, ax= plt.subplots(figsize=(10,5))

ax.plot(ts, h_td.strain/h_fd.snr, label='unlensed', c='C0', alpha=0.6)

ax.plot(ts, h_td_lens.strain/h_fd_lens.snr, label='lensed', c='C1', zorder=0)
ax.set_ylabel('strain')
xlim=(-Mtot/50, time_delay+Mtot/300)
ax.set_xlim(*xlim)

ax.set_xlabel('t [s]')
ax.grid(alpha=0.4)

ax.set_title('$M_{{\\rm BBH}}= {:s}\,M_\odot,\,M_{{\\rm lz}}={:s}\,M_\odot,\,y={:.1f}$'.format(tools.latex_float(Mtot),tools.latex_float(units_lens.Mlz.value),y))

ax.legend(ncols=4)

